In [1]:
import tensorflow as tf
import numpy as np

In [2]:
pc_init = np.array([[50.0, 0, 50.0]]).transpose()
CE = np.array([[0, 0, -50.0]]).transpose()

RCT = tf.placeholder(dtype=tf.float64, shape=(3, 3), name='CameraRotationTranspose')
PC = tf.Variable(initial_value=pc_init , name='CameraPosition')
CE = tf.Variable(initial_value=CE, name='CFrameViewPoint')
h = -CE[2]
points = tf.placeholder(dtype=tf.float64, shape=(3, 4))
im_points_targets = tf.placeholder(dtype=tf.float64, shape=(2, 4))

difference = points - PC

C_points = tf.matmul(RCT, difference)

coeff = h / (h + C_points[2, :])

im_points = C_points[:2, :] * coeff + (1 - coeff) * CE[:2]

In [3]:
loss_target = tf.reduce_mean(tf.squared_difference(im_points_targets, im_points))

In [4]:
learning_rate = tf.Variable(initial_value=0.1)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_target)

In [5]:
source = np.float32([[14.32 , 140.71], [ 120.78, 95.5], [199.49 ,96.84], [302.7 ,140.71]])
source = np.array([160 - source[:, 1], source[:, 0]])
source = source.transpose()

source = source.transpose()
FRAME_SHAPE = (160, 320)
FRAME_ORIGIN = (FRAME_SHAPE[1]/2, FRAME_SHAPE[0])
DST_SIZE = 5
BOTTOM_OFFSET = 6
WORLD_SIZE = 200
target = np.float32([[FRAME_SHAPE[1]/2 - DST_SIZE,
               FRAME_SHAPE[0] - BOTTOM_OFFSET, 0],
              [FRAME_SHAPE[1]/2 - DST_SIZE,
               FRAME_SHAPE[0] - BOTTOM_OFFSET - 2 * DST_SIZE, 0],
              [FRAME_SHAPE[1]/2 + DST_SIZE,
               FRAME_SHAPE[0] - BOTTOM_OFFSET - 2 * DST_SIZE, 0],
              [FRAME_SHAPE[1]/2 + DST_SIZE,
               FRAME_SHAPE[0] - BOTTOM_OFFSET, 0]])
target = target.transpose()

In [6]:
print(source)

[[  19.28999329   64.5          63.16000366   19.28999329]
 [  14.31999969  120.77999878  199.49000549  302.70001221]]


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
zero_rotation = np.array([[0, 0, 1], [0, -1, 0], [1, 0, 0]])
for i in range(500000):
    if i % 40000 == 0:
        sess.run(learning_rate.assign(learning_rate / 2))
    _, loss = sess.run([train_op, loss_target], 
                   feed_dict={points: target, im_points_targets: 
                              source, RCT: zero_rotation})
    if i % 5000 == 0:
        print('current loss: ', loss)

current loss:  28538.00612
current loss:  5779.47118424
current loss:  5765.84460242
current loss:  5751.54125154
current loss:  5736.35974619
current loss:  5720.05239979
current loss:  5702.3047533
current loss:  5682.70425705
current loss:  5660.69014501
current loss:  5648.54252756
current loss:  5635.46889216
current loss:  5621.30916519
current loss:  5605.8632643
current loss:  5588.87680543
current loss:  5570.01997057
current loss:  5548.8552576
current loss:  5524.78638734
current loss:  5511.41334703
current loss:  5496.97353017
current loss:  5481.299859
current loss:  5464.18541899
current loss:  5445.37010894
current loss:  5424.52130847
current loss:  5401.20507903
current loss:  5374.84182105
current loss:  5360.2799173
current loss:  5344.63515216
current loss:  5327.75473879
current loss:  5309.45300776
current loss:  5289.5015333
current loss:  5267.61532735
current loss:  5243.43308654
current loss:  5216.48817462
current loss:  5201.79467697
current loss:  5186.164

KeyboardInterrupt: 

In [11]:
source

array([[  14.31999969,  120.77999878,  199.49000549,  302.70001221],
       [ 140.71000671,   95.5       ,   96.83999634,  140.71000671],
       [   0.        ,    0.        ,    0.        ,    0.        ]], dtype=float32)

In [9]:
target

array([[ 155.,  155.,  165.,  165.],
       [ 154.,  144.,  144.,  154.],
       [   0.,    0.,    0.,    0.]], dtype=float32)

In [10]:
print(target)

[[ 155.  155.  165.  165.]
 [ 154.  144.  144.  154.]
 [   0.    0.    0.    0.]]
